PARAGON GEEKS — AI REPAIR ASSISTANT
Hybrid Rule-Based Operational Triage Engine

This project models a structured intake decision system for a real electronics repair business (Paragon Geeks).

The assistant simulates front-desk triage using a weighted rule-based classification engine designed to standardize intake decisions.

🎯 Business Objective

Repair intake can be inconsistent and subjective.
This system formalizes the intake process by:

- Categorizing incoming device issues

- Detecting multi-issue scenarios

- Escalating priority when complexity increases

- Estimating repair time expectations

- Routing ambiguous cases to diagnostics


In [56]:
# ============================================================
# REPAIR PLAN DEFINITIONS
# ============================================================

REPAIR_PLANS = {

    "Screen Repair": {
        "desc": "Display damage or failure (cracks, lines, flickering, black screen, touch failure).",
        "severity": "High",
        "duration": "10–15 minutes"
    },

    "Battery Replacement": {
        "desc": "Rapid drain, shutdowns, overheating, swelling, or poor battery performance.",
        "severity": "Medium",
        "duration": "10–15 minutes"
    },

    "Charging Port Repair": {
        "desc": "Intermittent charging or charging only at angle; likely port contamination or failure.",
        "severity": "Medium",
        "duration": "15-45 minutes"
    },

    "Water Damage Treatment": {
        "desc": "Liquid exposure; corrosion risk requiring internal inspection and cleaning.",
        "severity": "Critical",
        "duration": "Same-day to 1–3 days"
    },

    "Speaker / Mic Repair": {
        "desc": "Muffled audio, no sound, microphone not picking up voice.",
        "severity": "Low–Medium",
        "duration": "10–20 minutes"
    },

    "Software / Data Issue": {
        "desc": "Boot loop, OS crash, update failure, restore issues, severe lag.",
        "severity": "Medium",
        "duration": "30–120 minutes"
    },

    "Console / HDMI Repair": {
        "desc": "No signal or HDMI port failure on console.",
        "severity": "Medium",
        "duration": "Same-day to 1–3 days"
    },

    "Computer / Laptop Repair": {
        "desc": "Laptop/desktop hardware issues (black screen, boot failure, board fault).",
        "severity": "Medium–High",
        "duration": "Same-day to multi-day"
    },

    "Diagnostic / General Check": {
        "desc": "Unclear issue requiring technician diagnosis.",
        "severity": "Unknown",
        "duration": "15–30 minutes"
    }
}


In [71]:
def infer_device_type(device_name):
    name = device_name.lower()

    if any(k in name for k in [
        "iphone", "galaxy", "pixel", "s22", "s23", "s24",
        "ultra", "oneplus", "motorola", "moto", "pixel", "samsung"
    ]):
        return "phone"

    if any(k in name for k in [
        "ipad", "tablet"
    ]):
        return "tablet"

    if any(k in name for k in [
        "macbook", "imac", "laptop", "computer", "hp", "lenovo", "dell", "asus", "acer"
    ]):
        return "computer"

    if any(k in name for k in [
        "ps5", "ps4", "xbox", "console", "sony", "microsoft"
    ]):
        return "console"

    return "unknown"


hybrid triange logic

In [100]:
# =====================================================
# PARAGON GEEKS — HYBRID TRIAGE ENGINE (PORTFOLIO FINAL)
# =====================================================

# ===============================
# KEYWORD LIBRARIES
# ===============================

WATER_KEYWORDS = [
    "water", "liquid", "spilled", "coffee",
    "fell in water", "wet", "moisture"
]

CAMERA_KEYWORDS = [
    "camera", "rear camera",
    "camera blurry", "blurry",
    "wont focus", "won't focus",
    "camera glass", "lens cracked",
    "distorted image"
]

SCREEN_KEYWORDS = [
    "screen", "display",
    "crack", "cracked", "shattered",
    "touch not working",
    "dead spot", "dead spots",
    "flicker", "flickering",
    "green line", "lines on display",
    "black screen", "screen black",
    "no display", "display not working",
    "lcd", "oled", "digitizer"
]

BATTERY_KEYWORDS = [
    "battery health", "battery low",
    "drains", "dies fast",
    "overheating", "getting hot",
    "shutting off", "random shutdown",
    "won't hold charge", "wont hold charge"
]

CHARGING_KEYWORDS = [
    "hold cable", "wiggle", "angle",
    "loose port", "not charging",
    "only charges if", "charging port"
]

SOFTWARE_KEYWORDS = [
    "boot loop", "stuck in boot",
    "after update", "apps crashing",
    "laggy", "software issue"
]

BOARD_KEYWORDS = [
    "no signs of life",
    "won't turn on", "wont turn on",
    "wont power", "not powering on",
    "dead phone", "random restart",
    "random restarts"
]


# ===============================
# DEVICE TYPE INFERENCE
# ===============================

def infer_device_type(device_name):
    name = device_name.lower()

    if any(word in name for word in ["iphone", "galaxy", "pixel"]):
        return "phone"
    if any(word in name for word in ["ipad", "tablet"]):
        return "tablet"
    if any(word in name for word in ["macbook", "laptop"]):
        return "computer"
    if any(word in name for word in ["ps5", "xbox"]):
        return "console"

    return "unknown"


# ===============================
# PRIORITY LOGIC
# ===============================

def determine_priority(category):
    if category in ["Water Damage Treatment", "Board-Level / No Power"]:
        return "High"
    if category in ["Screen Repair", "Charging Port Repair", "Battery Replacement"]:
        return "Medium"
    return "Low"


# ===============================
# TIME ESTIMATION
# ===============================

def estimate_time(category):
    if category == "Screen Repair":
        return "45–60 minutes"
    if category == "Battery Replacement":
        return "30–45 minutes"
    if category == "Charging Port Repair":
        return "60–90 minutes"
    if category == "Water Damage Treatment":
        return "24–48 hour diagnostic cycle"
    if category == "Board-Level / No Power":
        return "Advanced diagnostic required (1–3 days)"
    if category == "Camera Repair":
        return "45–60 minutes"
    if category == "Software / Data Issue":
        return "30–90 minutes"
    if category == "Console / HDMI Repair":
        return "1–3 days"
    return "Varies"


# ===============================
# ROUTE ISSUE ENGINE
# ===============================
# ===============================
# ROUTE ISSUE ENGINE (WEIGHTED VERSION)
# ===============================

def route_issue(issue, device_type):

    issue = issue.lower()

    # Console override first
    if device_type == "console":
        if "no signal" in issue or "nothing shows" in issue or "hdmi" in issue:
            category = "Console / HDMI Repair"
            return build_output(category)

    category_scores = {
        "Water Damage Treatment": sum(word in issue for word in WATER_KEYWORDS),
        "Camera Repair": sum(word in issue for word in CAMERA_KEYWORDS),
        "Screen Repair": sum(word in issue for word in SCREEN_KEYWORDS),
        "Battery Replacement": sum(word in issue for word in BATTERY_KEYWORDS),
        "Charging Port Repair": sum(word in issue for word in CHARGING_KEYWORDS),
        "Software / Data Issue": sum(word in issue for word in SOFTWARE_KEYWORDS),
        "Board-Level / No Power": sum(word in issue for word in BOARD_KEYWORDS),
    }

    # Determine primary category
    primary_category = max(category_scores, key=category_scores.get)
    primary_score = category_scores[primary_category]

    # Detect secondary issues
    secondary_flags = [
        cat for cat, score in category_scores.items()
        if score > 0 and cat != primary_category
    ]

    # If nothing matched
    if primary_score == 0:
        category = "Diagnostic / General Check"
        return build_output(category)

    # Escalate priority if multiple issues detected
    priority = "High" if len(secondary_flags) > 0 else determine_priority(primary_category)

    return {
        "category": primary_category,
        "secondary_flags": secondary_flags,
        "confidence": min(0.95, 0.7 + 0.1 * primary_score),
        "priority": priority,
        "estimated_time": estimate_time(primary_category)
    }

# ===============================
# ROUTE ISSUE ENGINE (WEIGHTED VERSION)
# ===============================

def route_issue(issue, device_type):

    issue = issue.lower()

    # Console override first
    if device_type == "console":
        if "no signal" in issue or "nothing shows" in issue or "hdmi" in issue:
            category = "Console / HDMI Repair"
            return build_output(category)

    category_scores = {
        "Water Damage Treatment": sum(word in issue for word in WATER_KEYWORDS),
        "Camera Repair": sum(word in issue for word in CAMERA_KEYWORDS),
        "Screen Repair": sum(word in issue for word in SCREEN_KEYWORDS),
        "Battery Replacement": sum(word in issue for word in BATTERY_KEYWORDS),
        "Charging Port Repair": sum(word in issue for word in CHARGING_KEYWORDS),
        "Software / Data Issue": sum(word in issue for word in SOFTWARE_KEYWORDS),
        "Board-Level / No Power": sum(word in issue for word in BOARD_KEYWORDS),
    }

    # Determine primary category
    primary_category = max(category_scores, key=category_scores.get)
    primary_score = category_scores[primary_category]

    # Detect secondary issues
    secondary_flags = [
        cat for cat, score in category_scores.items()
        if score > 0 and cat != primary_category
    ]

    # If nothing matched
    if primary_score == 0:
        category = "Diagnostic / General Check"
        return build_output(category)

    # Escalate priority if multiple issues detected
    priority = "High" if len(secondary_flags) > 0 else determine_priority(primary_category)

    return {
        "category": primary_category,
        "secondary_flags": secondary_flags,
        "confidence": min(0.95, 0.7 + 0.1 * primary_score),
        "priority": priority,
        "estimated_time": estimate_time(primary_category)
    }



# ===============================
# OUTPUT BUILDER
# ===============================

def build_output(category):

    confidence_map = {
        "Water Damage Treatment": 0.95,
        "Board-Level / No Power": 0.85,
        "Screen Repair": 0.90,
        "Battery Replacement": 0.85,
        "Charging Port Repair": 0.88,
        "Camera Repair": 0.80,
        "Software / Data Issue": 0.80,
        "Console / HDMI Repair": 0.85,
        "Diagnostic / General Check": 0.40
    }

    return {
        "category": category,
        "confidence": confidence_map.get(category, 0.75),
        "priority": determine_priority(category),
        "estimated_time": estimate_time(category)
    }


demoset

In [110]:
# ==========================================================
# DEMO DATA (GLOBAL)
# ==========================================================

from collections import Counter

DEMO_CASES = [
    ("iphone 14 pro max", "dropped it and now screen is cracked and touch not working"),
    ("iphone 13", "green lines on display after dropping"),
    ("galaxy s23 ultra", "screen flickering and has dead spots"),
    ("iphone xr", "screen completely black but phone still vibrates"),
    ("iphone 11", "battery dies fast unless plugged in"),
    ("galaxy s22 ultra", "phone getting hot and shutting off"),
    ("iphone se", "battery health low and drains quickly"),
    ("iphone 12", "only charges if i hold the cable at an angle"),
    ("pixel 7a", "charging port feels loose and not charging"),
    ("ipad 9", "wont charge unless i wiggle charger"),
    ("iphone 15 pro max", "fell in water and now screen flickers"),
    ("macbook", "spilled coffee on keyboard and now it wont power on"),
    ("iphone 14", "rear camera is blurry and wont focus"),
    ("galaxy z flip", "camera glass cracked and image distorted"),
    ("iphone 13 pro max", "phone wont turn on at all no signs of life"),
    ("ipad pro", "random restarts and wont power properly"),
    ("iphone 12 pro max", "stuck in boot loop after update"),
    ("galaxy s24 ultra", "apps crashing and laggy after update"),
    ("ps5", "no signal on tv and hdmi not working"),
    ("xbox series x", "console turns on but nothing shows on screen")
]

# ==========================================================
# AUTO DEMO RUN
# ==========================================================

def run_demo_cases():

    print("\n================ AUTO DEMO RUN ================\n")

    category_results = []

    for device_name, issue in DEMO_CASES:

        device_type = infer_device_type(device_name)
        result = route_issue(issue, device_type)

        category_results.append(result["category"])

        print("------------------------------------------------------------")
        print(f"Device: {device_name} ({device_type})")
        print(f"Issue:  {issue}")
        print("Category:", result["category"])
        print("Confidence:", round(result["confidence"], 2))
        print("Priority:", result["priority"])
        print("Estimated Time:", result["estimated_time"])
        print("Secondary Flags:", result.get("secondary_flags", []))
        print("------------------------------------------------------------\n")

    print("=========== CATEGORY DISTRIBUTION ===========")
    print(Counter(category_results))
    print("\n================ END AUTO DEMO ================\n")


# ==========================================================
# LIVE INTERACTIVE MODE
# ==========================================================

def run_interactive_mode():

    print("\n================ LIVE TRIAGE MODE =================")
    print("Type 'exit' to stop.\n")

    while True:

        device_name = input("Enter device name: ").strip()
        if device_name.lower() == "exit":
            break

        issue = input("Describe the issue: ").strip()
        if issue.lower() == "exit":
            break

        device_type = infer_device_type(device_name)
        result = route_issue(issue, device_type)

        print("\n--- TRIAGE RESULT ---")
        print("Category:", result["category"])
        print("Confidence:", round(result["confidence"], 2))
        print("Priority:", result["priority"])
        print("Estimated Time:", result["estimated_time"])
        print("Secondary Flags:", result.get("secondary_flags", []))
        print("----------------------\n")

    print("\nSystem Status: Operational ✔\n")


# ==========================================================
# EXECUTION
# ==========================================================

if __name__ == "__main__":
    run_demo_cases()
    run_interactive_mode()



================ AUTO DEMO RUN ================

------------------------------------------------------------
Device: iphone 14 pro max (phone)
Issue:  dropped it and now screen is cracked and touch not working
Category: Screen Repair
Confidence: 0.95
Priority: Medium
Estimated Time: 45–60 minutes
Secondary Flags: []
------------------------------------------------------------

------------------------------------------------------------
Device: iphone 13 (phone)
Issue:  green lines on display after dropping
Category: Screen Repair
Confidence: 0.95
Priority: Medium
Estimated Time: 45–60 minutes
Secondary Flags: []
------------------------------------------------------------

------------------------------------------------------------
Device: galaxy s23 ultra (phone)
Issue:  screen flickering and has dead spots
Category: Screen Repair
Confidence: 0.95
Priority: Medium
Estimated Time: 45–60 minutes
Secondary Flags: []
------------------------------------------------------------

------

Enter device name:  iphone 14
Describe the issue:  cracked screen and not charging 



--- TRIAGE RESULT ---
Category: Screen Repair
Confidence: 0.95
Priority: High
Estimated Time: 45–60 minutes
Secondary Flags: ['Charging Port Repair']
----------------------



Enter device name:  exit



System Status: Operational ✔



📈 Operational Impact & Future Deployment

This system demonstrates:

- Business rule modeling

- Structured classification design

- Decision automation logic

- Escalation architecture

- Domain-driven AI design

In a production environment, this could be extended into:

- A web-based intake portal (Streamlit)

- POS integration logging

- Repair analytics dashboards

- Intake data warehousing

- Operational forecasting models

The current implementation focuses on logic validation and decision modeling.